In [1]:
from data_sampler import WeightedSampler
from dataset import CARLADataset, CARLADatasetMultiProcessing
from data_preprocessing import preprocessing
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import torch
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path_ege_data = os.path.join("..", "data", "Dataset Ege")

config = {"used_inputs": ["rgb", "depth", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle"],
        "seq_len": 2
        }

dataset = CARLADataset(root_dir=path_ege_data, config=config)
dataset_mp = CARLADatasetMultiProcessing(root_dir=path_ege_data, config=config)
weighted_sampler = WeightedSampler(dataset=dataset)
print(dataset.__len__())

217


In [3]:
dl = DataLoader(dataset=dataset, batch_size=16, num_workers=0, sampler=weighted_sampler)
dl_mp = DataLoader(dataset=dataset_mp, batch_size=16, num_workers=3, sampler=weighted_sampler)

In [4]:
count = 0
for batch in dl:
    # preprocessing
    for key in preprocessing:
        batch[key] = preprocessing[key](batch[key])
    count += 1
print(count)

14


In [37]:
# Derive statistics for our dataset that can be used in Normalize() method to get approx. zero mean & var 1. 
mean = 0
std = 0
count = 0
for batch in dl:
    mean = (count * mean + batch["rgb"].shape[0] * batch["rgb"].mean(axis=[0, 1, 3, 4])) / (count + batch["rgb"].shape[0])
    std = (count * std + batch["rgb"].shape[0] * batch["rgb"].std(axis=[0, 1, 3, 4])) / (count + batch["rgb"].shape[0])
    count += batch["rgb"].shape[0]
print(count)

217


In [38]:
mean

tensor([60.1183, 70.7931, 89.4762], dtype=torch.float64)

In [34]:
std

tensor([49.3769, 49.9194, 56.3327], dtype=torch.float64)

In [27]:
batch["rgb"].mean(axis=[0, 1, 3, 4])

tensor([58.9700, 71.0216, 82.7724], dtype=torch.float64)

In [9]:
batch["rgb"].mean(axis=[0, 1, 3, 4])

tensor([61.5390, 71.6320, 91.3989], dtype=torch.float64)

In [ ]:
plt.imshow(preprocess(batch["rgb"][5, 0]).reshape(224, 224, 3)/255)

In [11]:
arr = np.random.random((200))

In [12]:
arr.mean()

0.4624097669144409

In [39]:
mean = 0
count = 0
for a in arr:
    mean = (count * mean + a) / (count + 1)
    count += 1

In [40]:
mean

0.46240976691444075